### Using Various APIs

In [1]:
!pip install flask
!pip install sqlalchemy
!pip install web3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.3 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 5.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.5/515.5 kB 4.8 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of eth-account to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of eth-rlp to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [289]:
import requests
from sqlalchemy import create_engine
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

# to_date = datetime.now()
# from_date = to_date - relativedelta(years=1)

# print(to_date)
# print(from_date)

def convert_unix(date):
    dt = datetime.strptime(date, '%Y-%m-%d')
    return int(dt.timestamp())

def unix_to_datetime_days(dates):
    timestamp = int(dates) / 1000
    return datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d')

def unix_to_datetime_time(dates):
    timestamp = int(dates) / 1000
    return datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

def get_date_range(years):
    to_date = datetime.now()
    from_date = to_date - relativedelta(days=years*365)
    return from_date.strftime('%Y-%m-%d')

def fetch_historical_prices(coin_id, start_date, end_date):
    url = f'https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart/range'
    params = {
        'vs_currency': 'usd',
        'from': start_date,
        'to': end_date
    }
    response = requests.get(url, params=params)
    return response.json()

from_date = get_date_range(1)
# print(convert_unix(datetime.now().strftime('%Y-%m-%d')))
# print(convert_unix(from_date))
data = fetch_historical_prices('ethereum', convert_unix(from_date), convert_unix(datetime.now().strftime('%Y-%m-%d')))

# prices = []

prices_data = data['prices']

# for x, price in prices_data:
#     prices.append(price)

# for price in prices:
#     print(price)

market_cap_data = data['market_caps']
volumes_data = data['total_volumes']

for item in prices_data:
    item[0] = unix_to_datetime_days(item[0])

for item in market_cap_data:
    item[0] = unix_to_datetime_days(item[0])

for item in volumes_data:
    item[0] = unix_to_datetime_days(item[0])

prices_df = pd.DataFrame(prices_data, columns=['timestamp', 'price'])
market_cap_df = pd.DataFrame(market_cap_data, columns=['timestamp', 'market_caps'])
volumes_df = pd.DataFrame(volumes_data, columns=['timestamp', 'total_volumes'])

first_merge_df = pd.merge(prices_df, market_cap_df, on='timestamp')
df_1 = pd.merge(first_merge_df, volumes_df, on='timestamp')

df_1


/var/folders/vs/n4ky3gts21v7jxpqgmgdvb1h0000gn/T/ipykernel_24669/634111179.py:19: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  return datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d')


,timestamp,price,market_caps,total_volumes
0,2023-06-17,1716.377281,2.063216e+11,7.008800e+09
1,2023-06-18,1726.372810,2.075960e+11,4.859691e+09
2,2023-06-19,1719.273162,2.067308e+11,4.084739e+09
3,2023-06-20,1734.793103,2.085076e+11,5.770984e+09
4,2023-06-21,1790.190441,2.150610e+11,7.900259e+09
...,...,...,...,...
360,2024-06-11,3666.827665,4.405061e+11,8.786881e+09
361,2024-06-12,3498.555954,4.202517e+11,1.962193e+10
362,2024-06-13,3559.447534,4.278342e+11,1.679807e+10
363,2024-06-14,3465.316584,4.171515e+11,1.425759e+10


In [294]:
import numpy as np

df_1['daily_return'] = df_1['price'].pct_change()

# Calculate Volatility
volatility = np.std(df_1['daily_return'])

# Calculate Sharpe Ratio (Assuming Risk-Free Rate = 0%)
avg_daily_return = np.mean(df_1['daily_return'])
sharpe_ratio = avg_daily_return / volatility

# Calculate Drawdown
cumulative_return = np.cumprod(1 + df_1['daily_return']) - 1
previous_peaks = np.maximum.accumulate(cumulative_return)

start_idx = np.where(~np.isnan(cumulative_return))[0][0]

# Calculate Drawdown starting from the first valid index
drawdowns = (cumulative_return[start_idx:] - np.maximum.accumulate(cumulative_return[start_idx:])) / np.maximum.accumulate(cumulative_return[start_idx:])

covariance = df_1['daily_return'].cov()

print(covariance)

print("Volatility:", volatility)
print("Sharpe Ratio:", sharpe_ratio)
print("Drawdowns:", drawdowns)

df

TypeError: Series.cov() missing 1 required positional argument: 'other'

In [279]:
import requests

def fetch_ohlc(coin_id, days):
    params = {
        'vs_currency': 'usd',
        'days': days,
        'precision': 3
    }
    response = requests.get(f'https://api.coingecko.com/api/v3/coins/{coin_id}/ohlc', params=params)
    data = response.json()
    df_2 = pd.DataFrame([(item[0], item[-1]) for item in data], columns=['timestamp', 'close'])
    df_2['timestamp'] = df_2['timestamp'].apply(unix_to_datetime_time)
    return df_2

# def fetch_ohlc_repeat(coin_id):
#     # stupid granularity
#     splits = [20] * 19
#     for split in splits:
#         data.append(fetch_ohlc(coin_id=coin_id, days=split))
#     return pd.DataFrame([(item[0], item[-1]) for item in data], columns=['timestamp', 'close'])

# Valid values: 1, 7, 14, 30, 90, 180, 365

# fetch_ohlc('ethereum', days=2) # Intraday, 1 - 2
# fetch_ohlc('ethereum', days=14) # Swing, 7 - 14
fetch_ohlc('ethereum', days=90) # Long term, 30 - 90

/var/folders/vs/n4ky3gts21v7jxpqgmgdvb1h0000gn/T/ipykernel_24669/634111179.py:23: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  return datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')


,timestamp,close
0,2024-03-18 00:00:00,3643.280
1,2024-03-22 00:00:00,3493.430
2,2024-03-26 00:00:00,3588.493
3,2024-03-30 00:00:00,3516.096
4,2024-04-03 00:00:00,3274.901
5,2024-04-07 00:00:00,3362.839
6,2024-04-11 00:00:00,3539.763
7,2024-04-15 00:00:00,3157.684
8,2024-04-19 00:00:00,3064.912
9,2024-04-23 00:00:00,3199.770


In [268]:
import requests

def get_aave_lending_rates():
    url = "https://api.thegraph.com/subgraphs/name/aave/protocol-v2"
    query = """
    {
      reserves {
        symbol
        liquidityRate
        variableBorrowRate
        stableBorrowRate
      }
    }
    """
    response = requests.post(url, json={'query': query})
    data = response.json()

    if 'errors' in data:
        print("Error fetching data:", data['errors'])
        return

    reserves = data['data']['reserves']
    rates = {reserve['symbol']: reserve for reserve in reserves}
    return rates

lending_rates = get_aave_lending_rates()
for symbol, rates in lending_rates.items():
    if symbol == 'WETH':
        data = {
            'Cryptocurrency': [symbol],
            'Liquidity Rate': [int(rates['liquidityRate']) / 1e27 * 100],
            'Variable Borrow Rate': [int(rates['variableBorrowRate']) / 1e27 * 100],
            'Stable Borrow Rate': [int(rates['stableBorrowRate']) / 1e27 * 100]
        }
        # df_2 = pd.DataFrame([(item[0], item[-1]) for item in data], columns=['timestamp', 'close'])
        df_3 = pd.DataFrame(data)
  
df_3


,Cryptocurrency,Liquidity Rate,Variable Borrow Rate,Stable Borrow Rate
0,WETH,0.288904,2.316465,4.385753


In [269]:
import requests

def get_eth_circulating_supply():
    url = 'https://api.coingecko.com/api/v3/coins/ethereum'
    params = {
        'localization': 'false',
        'tickers': 'false',
        'market_data': 'true',
        'community_data': 'false',
        'developer_data': 'false',
        'sparkline': 'false'
    }
    response = requests.get(url, params=params)
    data = response.json()
    circulating_supply = data['market_data']['circulating_supply']
    return circulating_supply

circulating_supply = get_eth_circulating_supply()

print(f"Circulating Supply of ETH: {circulating_supply}")

data = {
    'Cryptocurrency': "ETH",
    'Circulating Supply': circulating_supply
}

new_row = pd.DataFrame([data])

df_3 = pd.concat([df_3, new_row], ignore_index=True)

df_3 = df_3.fillna("")

df_3


Circulating Supply of ETH: 120150089.71059


,Cryptocurrency,Liquidity Rate,Variable Borrow Rate,Stable Borrow Rate,Circulating Supply
0,WETH,0.288904,2.316465,4.385753,
1,ETH,,,,120150089.71059


In [272]:
import requests

def get_eth_tvl():
    url = 'https://api.llama.fi/chains'
    response = requests.get(url)
    # print(f"Response status code: {response.status_code}")
    # print(f"Response content: {response.text}") 
    response.raise_for_status()  
    data = response.json()
    
    eth_data = None
    for item in data:
        if item['name'].lower() == 'ethereum':
            eth_data = item
            break
    if eth_data:
        return eth_data['tvl']
    else:
        print("Ethereum data not found in the response.")

eth_tvl = get_eth_tvl()
print(f"The Total Value Locked (TVL) of Ethereum is ${eth_tvl}")

df_3['TVL'] = eth_tvl

df_3.loc[0, 'TVL'] = ""

df_3


The Total Value Locked (TVL) of Ethereum is $135527437953.96303


/var/folders/vs/n4ky3gts21v7jxpqgmgdvb1h0000gn/T/ipykernel_24669/1230244412.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_3.loc[0, 'TVL'] = ""


,Cryptocurrency,Liquidity Rate,Variable Borrow Rate,Stable Borrow Rate,Circulating Supply,TVL
0,WETH,0.288904,2.316465,4.385753,,
1,ETH,,,,120150089.71059,135527437953.963028


In [273]:
import requests

url = 'https://gas.mycryptoapi.com/'

try:
    response = requests.get(url)
    response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)

    data = response.json()

    print(f"Low gas price: {data['safeLow']} Gwei")
    print(f"Average gas price: {data['standard']} Gwei")
    print(f"Fast gas price: {data['fast']} Gwei")

except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
except requests.exceptions.RequestException as req_err:
    print(f"Request error occurred: {req_err}")
except ValueError as json_err:
    print(f"JSON decode error: {json_err}")

df_3['Low gas price'] = f"{data['safeLow']} Gwei"
df_3['Average gas price'] = f"{data['standard']} Gwei"
df_3['Fast gas price'] = f"{data['fast']} Gwei"

df_3.loc[0, 'Low gas price'] = ""
df_3.loc[0, 'Average gas price'] = ""
df_3.loc[0, 'Fast gas price'] = ""


df_3


Low gas price: 21 Gwei
Average gas price: 22 Gwei
Fast gas price: 34 Gwei


,Cryptocurrency,Liquidity Rate,Variable Borrow Rate,Stable Borrow Rate,Circulating Supply,TVL,Low gas price,Average gas price,Fast gas price
0,WETH,0.288904,2.316465,4.385753,,,,,
1,ETH,,,,120150089.71059,135527437953.963028,21 Gwei,22 Gwei,34 Gwei


In [274]:
import requests

def get_eth_transaction_volume():
    url = 'https://api.coingecko.com/api/v3/coins/ethereum/market_chart'
    params = {
        'vs_currency': 'usd',
        'days': '1',
        'interval': 'daily'
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        transaction_volumes = data['total_volumes']
        daily_volumes = [volume[1] for volume in transaction_volumes]
        total_volume = sum(daily_volumes)
        return total_volume
    else:
        print(f"Failed to fetch data: {response.status_code}")
        return None

volume = get_eth_transaction_volume()
if volume:
    print(f"Daily transaction volume of Ethereum: {volume} USD")

df_3['Daily transaction volume'] = volume

df_3.loc[0, 'Daily transaction volume'] = ""

df_3

Daily transaction volume of Ethereum: 28741391006.09904 USD


/var/folders/vs/n4ky3gts21v7jxpqgmgdvb1h0000gn/T/ipykernel_24669/180060427.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_3.loc[0, 'Daily transaction volume'] = ""


,Cryptocurrency,Liquidity Rate,Variable Borrow Rate,Stable Borrow Rate,Circulating Supply,TVL,Low gas price,Average gas price,Fast gas price,Daily transaction volume
0,WETH,0.288904,2.316465,4.385753,,,,,,
1,ETH,,,,120150089.71059,135527437953.963028,21 Gwei,22 Gwei,34 Gwei,28741391006.099041


In [241]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

# Define the Blockchair API endpoint for Ethereum
api_url = 'https://api.blockchair.com/ethereum/stats'

# Fetch data from Blockchair API
response = requests.get(api_url)
data = response.json()

# Print the entire response to understand its structure
print(data)

# Extract necessary fields with error handling
transaction_volume = data['data'].get('transactions_24h', 'N/A')
holding_addresses = data['data'].get('hodling_addresses', 'N/A')

# Print extracted data
print(f"Transaction Volume (24h): {transaction_volume}")
print(f"Holding Addresses: {holding_addresses}")

# Check if values are available before plotting
if transaction_volume != 'N/A' and holding_addresses != 'N/A':
    # Create a DataFrame for visualization (if you have historical data)
    df = pd.DataFrame({
        'timestamp': [pd.Timestamp.now()],
        'transaction_volume': [transaction_volume],
        'holding_addresses': [holding_addresses]
    })


{'data': {'blocks': 20095196, 'transactions': 2405763655, 'blocks_24h': 7162, 'circulation_approximate': '122375301624480000000000000', 'transactions_24h': 1191701, 'difficulty': 0, 'volume_24h_approximate': '1755542168608154600000000', 'mempool_transactions': 5345, 'mempool_median_gas_price': 2360010000, 'mempool_tps': 7.25, 'mempool_total_value_approximate': '70367017015789320000', 'best_block_height': 20094378, 'best_block_hash': 'eb17723cf93d737e6c4fba3a34bc41e7eb36bf03b1692bb97c4ae8db69718fad', 'best_block_time': '2024-06-15 02:59:11', 'uncles': 1306731, 'uncles_24h': 0, 'blockchain_size': 1035083764673, 'calls': 9596343577, 'average_transaction_fee_24h': '1145944666982714', 'median_transaction_fee_24h': '432394300380795', 'inflation_24h': 0, 'average_simple_transaction_fee_24h': '238279745342057', 'median_simple_transaction_fee_24h': '192941411932500', 'burned': '4327276478847316289133235', 'burned_24h': '972846416896017184309', 'largest_transaction_24h': {'hash': '0x97ec30ad2147